In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
from pyrosm import OSM, get_data
import geopandas as gpd
import pandas as pd
from sklearn.neighbors import BallTree
import numpy as np
import mapclassify as mc
# import matplotlib.pyplot as plt
import time
import networkx as nx
import igraph as ig
import osmnx as ox
import folium
import json
from datetime import datetime, timedelta
from folium import PolyLine, Marker
from folium.plugins import BeautifyIcon
from math import radians, cos, sin, asin, sqrt
import random as r
import math


In [3]:
# Coordenadas para Santiago Centro
lat, lon = -33.456574386393314, -70.65970917320084

# Ajusta los límites usando el valor de delta
delta = 0.09  # Menor valor para un área más pequeña
north = lat + delta
south = lat - delta
east = lon + delta
west = lon - delta

# Obtiene el grafo usando los límites ajustados
G = ox.graph_from_bbox(north, south, east, west, network_type='drive')
G = ox.speed.add_edge_speeds(G)
G = ox.speed.add_edge_travel_times(G)

almacen = {"id": "0", "lat": "-33.456574386393314", "long": "-70.65970917320084", "demand": 0}

# Datos de los clientes
with open("nodos.json", 'r', encoding='utf-8') as archivo:
    nodos = json.load(archivo)


nodos.insert(0, almacen)  # Asegúrate de que el almacén es el primer nodo en la lista
# r.seed(42)
# np.random.seed(42)

In [4]:
def distancia_camion(nodo1, nodo2, G):
    source_node = ox.distance.nearest_nodes(G, X=float(nodo1['long']), Y=float(nodo1['lat']))
    target_node = ox.distance.nearest_nodes(G, X=float(nodo2['long']), Y=float(nodo2['lat']))
    ruta = ox.shortest_path(G, source_node, target_node, weight='length')
    return sum(ox.utils_graph.get_route_edge_attributes(G, ruta, 'length'))

def haversine(lat1, lon1, lat2, lon2):
    # Convertir coordenadas de grados a radianes
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Fórmula de Haversine
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371 # Radio de la Tierra en kilómetros
    return c * r

def distancia_dron(nodo1, nodo2):
    return haversine(float(nodo1['lat']), float(nodo1['long']), float(nodo2['lat']), float(nodo2['long']))

def matriz_distancias_camion(lista_nodos, G):
    cant_nodos = len(lista_nodos)
    matriz_distancias_camion = [[0 for _ in range(cant_nodos)] for _ in range(cant_nodos)]
    for i in range(cant_nodos):
        for j in range(i + 1, cant_nodos):
            distancia = distancia_camion(lista_nodos[i], lista_nodos[j], G)
            matriz_distancias_camion[i][j] = matriz_distancias_camion[j][i] = distancia/1000
    return matriz_distancias_camion

def matriz_distancias_dron(lista_nodos):
    cant_nodos = len(lista_nodos)
    matriz_distancias_dron = [[0 for _ in range(cant_nodos)] for _ in range(cant_nodos)]
    for i in range(cant_nodos):
        for j in range(i + 1, cant_nodos):
            distancia = distancia_dron(lista_nodos[i], lista_nodos[j])
            matriz_distancias_dron[i][j] = matriz_distancias_dron[j][i] = distancia
    return matriz_distancias_dron

def calcular_matrices_costos(matriz_dist_camion, costo_camion, matriz_dist_dron, costo_dron):
    cant_nodos = len(matriz_dist_camion)  # Asumiendo que ambas matrices tienen el mismo tamaño
    matriz_costos_camion = [[0 for _ in range(cant_nodos)] for _ in range(cant_nodos)]
    matriz_costos_dron = [[0 for _ in range(cant_nodos)] for _ in range(cant_nodos)]

    for i in range(cant_nodos):
        for j in range(cant_nodos):
            matriz_costos_camion[i][j] = matriz_dist_camion[i][j] * costo_camion
            matriz_costos_dron[i][j] = matriz_dist_dron[i][j] * costo_dron

    return matriz_costos_camion, matriz_costos_dron




def calcular_matrices_tiempos(matriz_dist_camion, velocidad_camion, matriz_dist_dron, velocidad_dron):
    cant_nodos = len(matriz_dist_camion)
    matriz_tiempos_camion = [[0 for _ in range(cant_nodos)] for _ in range(cant_nodos)]
    matriz_tiempos_dron = [[0 for _ in range(cant_nodos)] for _ in range(cant_nodos)]

    for i in range(cant_nodos):
        for j in range(cant_nodos):
            if matriz_dist_camion[i][j] != 0:
                matriz_tiempos_camion[i][j] = (matriz_dist_camion[i][j] / velocidad_camion) * 3600
            if matriz_dist_dron[i][j] != 0:
                matriz_tiempos_dron[i][j] = (matriz_dist_dron[i][j] / velocidad_dron) * 3600

    return matriz_tiempos_camion, matriz_tiempos_dron

def matriz_feromonas(lista_nodos,f0):
    cant_nodos = len(lista_nodos)
    matriz_feromonas = []
    for i in range(cant_nodos):
        matriz_feromonas.append([0 if i == j else f0 for j in range(cant_nodos)])

    return matriz_feromonas


In [5]:
costo_camion = 1.0  # Ejemplo de costo por km para el camión
costo_dron = 0.5    # Ejemplo de costo por km para el dron
velocidad_camion = 50  # km/h
velocidad_dron = 40   # km/h
f0 = 1 # valor inicial feromonas, un valor random que sea con distribucion uniforme
matriz_dist_camion = matriz_distancias_camion(nodos, G)
matriz_dist_dron = matriz_distancias_dron(nodos)

# 
matriz_costos_camion, matriz_costos_dron = calcular_matrices_costos(
    matriz_dist_camion, costo_camion, matriz_dist_dron, costo_dron
)

matriz_tiempos_camion, matriz_tiempos_dron = calcular_matrices_tiempos(
    matriz_dist_camion, velocidad_camion, matriz_dist_dron, velocidad_dron
)
#libreria 
matriz_feromonas_camion = matriz_feromonas(nodos,f0)#camion  rutas mas cortas, probar que hace la ruta completa, le saco una cordenada y la calculo de nuevo, la complemento con el dron y le sumo el costo de mandar el dron ahi,
#despues le saco otra cordenada, que me devuelva cual es la cantidad en viajes de dron, que en base a eso me arme los csotos, la combinatoria en dron, combinatoria entera, cuantos viajes y cuales, me voy quedando con el menor cosd, un doble for con muchas funciones
matriz_fermonas_dron = matriz_feromonas(nodos,f0)# esta mas abierto no hay calles definidas


### Matriz de Distancias para el camión

In [6]:
matriz_dist_camion

[[0,
  0.639073,
  1.2250900000000002,
  1.8999140000000003,
  3.190099,
  2.7013080000000005,
  3.5835310000000002,
  4.319865000000001,
  4.32039,
  5.221424999999998,
  7.9926229999999965,
  7.3024640000000005,
  4.483667,
  5.143953000000002,
  3.8654990000000002,
  4.648547000000001,
  5.142725000000001,
  5.2295339999999975,
  4.636601999999999,
  3.561864,
  4.0816099999999995,
  7.8671479999999985,
  6.210045999999998,
  2.986738,
  3.8447169999999993,
  1.8811289999999998,
  2.8712460000000006,
  3.8285599999999995,
  8.877924000000002,
  2.2145639999999998,
  3.1732139999999998],
 [0.639073,
  0,
  0.5860169999999999,
  1.2608409999999999,
  2.575915,
  2.0622350000000003,
  2.944458,
  3.6807920000000003,
  3.6813170000000004,
  4.582351999999999,
  7.353549999999998,
  6.663391000000001,
  3.8445940000000003,
  4.50488,
  3.226426,
  4.009474,
  4.503651999999999,
  4.628799999999999,
  4.917146000000001,
  3.1049590000000005,
  4.433164999999999,
  7.228074999999999,
  5.5

### Matriz de Distancias para el Dron

In [7]:
matriz_dist_dron

[[0,
  0.6378866613994291,
  1.138960133178448,
  1.497803553951977,
  2.266280671631359,
  2.689472441327137,
  3.54026709513087,
  3.858044467455383,
  4.263205163476998,
  5.233680795520816,
  7.280841721775403,
  6.72398715165233,
  3.991222054809571,
  3.887896093733944,
  3.2517998532818884,
  3.22764032146678,
  3.501553404816594,
  3.7378988565405735,
  3.482262343847186,
  2.645482985691797,
  2.9984956127919165,
  7.4471829607169795,
  6.0411487509053075,
  2.478167240474966,
  2.522693326835378,
  1.1600317791839678,
  1.9348104751022503,
  2.4779668361354314,
  8.054062439725907,
  1.616801047259647,
  2.2381104545454713],
 [0.6378866613994291,
  0,
  0.5021203563687345,
  0.8733672685939564,
  1.8888899880100916,
  2.051860551440999,
  2.9027019694318215,
  3.2310724634184047,
  3.6277847338395324,
  4.602361273828643,
  6.643047019105883,
  6.096071568154001,
  3.358606833058467,
  3.322113052568872,
  2.6269237767886064,
  2.680414668328154,
  2.9963246406347324,
  3.297

### Matriz de Costos para el camión

In [8]:
matriz_costos_camion

[[0.0,
  0.639073,
  1.2250900000000002,
  1.8999140000000003,
  3.190099,
  2.7013080000000005,
  3.5835310000000002,
  4.319865000000001,
  4.32039,
  5.221424999999998,
  7.9926229999999965,
  7.3024640000000005,
  4.483667,
  5.143953000000002,
  3.8654990000000002,
  4.648547000000001,
  5.142725000000001,
  5.2295339999999975,
  4.636601999999999,
  3.561864,
  4.0816099999999995,
  7.8671479999999985,
  6.210045999999998,
  2.986738,
  3.8447169999999993,
  1.8811289999999998,
  2.8712460000000006,
  3.8285599999999995,
  8.877924000000002,
  2.2145639999999998,
  3.1732139999999998],
 [0.639073,
  0.0,
  0.5860169999999999,
  1.2608409999999999,
  2.575915,
  2.0622350000000003,
  2.944458,
  3.6807920000000003,
  3.6813170000000004,
  4.582351999999999,
  7.353549999999998,
  6.663391000000001,
  3.8445940000000003,
  4.50488,
  3.226426,
  4.009474,
  4.503651999999999,
  4.628799999999999,
  4.917146000000001,
  3.1049590000000005,
  4.433164999999999,
  7.228074999999999,
 

### Matriz de Costos para el Dron

In [9]:
matriz_costos_dron

[[0.0,
  0.31894333069971453,
  0.569480066589224,
  0.7489017769759885,
  1.1331403358156795,
  1.3447362206635685,
  1.770133547565435,
  1.9290222337276914,
  2.131602581738499,
  2.616840397760408,
  3.6404208608877013,
  3.361993575826165,
  1.9956110274047856,
  1.943948046866972,
  1.6258999266409442,
  1.61382016073339,
  1.750776702408297,
  1.8689494282702868,
  1.741131171923593,
  1.3227414928458985,
  1.4992478063959582,
  3.7235914803584897,
  3.0205743754526537,
  1.239083620237483,
  1.261346663417689,
  0.5800158895919839,
  0.9674052375511252,
  1.2389834180677157,
  4.027031219862954,
  0.8084005236298235,
  1.1190552272727357],
 [0.31894333069971453,
  0.0,
  0.25106017818436727,
  0.4366836342969782,
  0.9444449940050458,
  1.0259302757204996,
  1.4513509847159107,
  1.6155362317092024,
  1.8138923669197662,
  2.3011806369143213,
  3.3215235095529416,
  3.0480357840770007,
  1.6793034165292335,
  1.661056526284436,
  1.3134618883943032,
  1.340207334164077,
  1.498

### Matriz de tiempos para el camión

In [10]:
matriz_tiempos_camion

[[0,
  46.013256,
  88.20648000000001,
  136.79380800000004,
  229.68712799999997,
  194.49417600000004,
  258.014232,
  311.03028000000006,
  311.06807999999995,
  375.94259999999986,
  575.4688559999997,
  525.777408,
  322.82402399999995,
  370.3646160000002,
  278.315928,
  334.69538400000005,
  370.27620000000013,
  376.52644799999985,
  333.8353439999999,
  256.454208,
  293.87591999999995,
  566.4346559999999,
  447.1233119999999,
  215.04513599999999,
  276.819624,
  135.441288,
  206.72971200000003,
  275.65632,
  639.2105280000002,
  159.44860799999998,
  228.471408],
 [46.013256,
  0,
  42.193223999999994,
  90.78055199999999,
  185.46588,
  148.48092,
  212.000976,
  265.017024,
  265.05482400000005,
  329.92934399999996,
  529.4555999999998,
  479.764152,
  276.81076800000005,
  324.35136,
  232.30267200000003,
  288.682128,
  324.26294399999995,
  333.27359999999993,
  354.03451200000006,
  223.55704800000004,
  319.1878799999999,
  520.4214,
  401.1100559999999,
  235.24

### Matriz de tiempo para el dron

In [11]:
matriz_tiempos_dron

[[0,
  57.40979952594862,
  102.50641198606031,
  134.80231985567792,
  203.96526044682233,
  242.05251971944233,
  318.6240385617783,
  347.22400207098445,
  383.6884647129299,
  471.03127159687347,
  655.2757549597862,
  605.1588436487096,
  359.2099849328614,
  349.91064843605494,
  292.66198679536996,
  290.4876289320102,
  315.13980643349345,
  336.4108970886516,
  313.4036109462467,
  238.09346871226174,
  269.8646051512725,
  670.2464664645281,
  543.7033875814777,
  223.03505164274694,
  227.042399415184,
  104.40286012655712,
  174.13294275920254,
  223.01701525218883,
  724.8656195753317,
  145.51209425336822,
  201.4299409090924],
 [57.40979952594862,
  0,
  45.19083207318611,
  78.60305417345607,
  170.00009892090824,
  184.6674496296899,
  261.2431772488639,
  290.79652170765644,
  326.5006260455579,
  414.2125146445778,
  597.8742317195295,
  548.6464411338601,
  302.27461497526207,
  298.99017473119846,
  236.4231399109746,
  241.23732014953384,
  269.6692176571259,
  29

In [12]:
def nodos_disponibles_1a2f(lista_nodos,i1,i2,porVisitar,tiemposD,max_vuelo,capacidad_dron):#falta probar todas las combinacione sposibles 
    nodos_disponibles = []
    for j in porVisitar:
        if lista_nodos[j]['demand'] <= capacidad_dron and lista_nodos[j]['demand'] != 0:# limite de cuantos nodos puede visitar el dron, el limite max en base al json y cuantos a visitiado para poder hacer el if, falta 
            #comparar la combinatoria me quedo con la de menor costo, limitar la cantidad de visitar que vaya el dron
            if tiemposD[i1][j] + tiemposD[j][i2] < max_vuelo:
                nodos_disponibles.append(j)

    return nodos_disponibles 


def hormiga(lista_nodos,nodo_inicial,costosC,costosD,tiemposC,tiemposD,feromonasC,feromonasD,alfa,beta,const,max_vuelo,max_capacidad):
    cant_nodos = len(lista_nodos)
    ruta_camion = [nodo_inicial]
    ruta_dron = [nodo_inicial]
    porVisitar = [x for x in range(cant_nodos)]
    porVisitar.remove(nodo_inicial)
    i = nodo_inicial
    z = 0.5 #PARAMETRO AUN POR DEFINIR
    h = 0
    while porVisitar:
        h += 1
        #siguiente nodo camion
        suma_total = 0
        probabilidadesC = []
        for j in porVisitar:
            p = (feromonasC[i][j]**alfa)*((const/float(costosC[i][j]))**beta)
            probabilidadesC.append([j,p])
            suma_total += p
        r.seed()
        siguienteC = r.random()
        suma = 0
        for k in probabilidadesC:
            suma += k[1]
            if siguienteC <= suma/float(suma_total):
                porVisitar.remove(k[0])
                ruta_camion.append(k[0])
                break
        #siguiente nodo dron
        probabilidadesD = []
        randomZ = r.random()
        siguienteD = None
        nodos_disponibles = nodos_disponibles_1a2f(lista_nodos,i,ruta_camion[-1],porVisitar,tiemposD,max_vuelo,max_capacidad)
        if nodos_disponibles and randomZ < z: #como se utiliza las feromonasD quiza ya no sea necesario z
            suma_total = 0
            probabilidadesD = []
            for j in nodos_disponibles:
                p =  (feromonasC[i][j]**alfa)*((const/float(costosC[i][j]))**beta)
                probabilidadesD.append([j,p])
                suma_total += p
            r.seed()
            siguienteD = r.random()
            suma = 0
            for k in probabilidadesD:
                suma += k[1]
                if siguienteD <= suma/float(suma_total):
                    porVisitar.remove(k[0])
                    ruta_dron.append(k[0])
                    break
        ruta_dron.append(ruta_camion[-1])
        i = ruta_camion[-1]

    return  ruta_camion, ruta_dron


In [13]:
def max_feromonas(mejor_costo,evaporacion):
    return 1/float(evaporacion*mejor_costo)

def min_feromonas(max_feromonas,cant_nodos,mejor_costo):#sintonisacion de parametros
    raiz = math.factorial(cant_nodos)**(1/cant_nodos)
    return (max_feromonas*(1-raiz))/((cant_nodos/2-1)*raiz)


def costo_tiempo(ruta_camion,ruta_dron,costosC,costosD,tiemposC,tiemposD):#

    #nuevo: todas las psoibles rutas camiones y drones, y evluaar si es ruta valida , descartar las rutas invalidas, descartar la combinacion, y esas pasan a la evalucion
    #que el dron no sopbrepase distancia maxima
    
    costo = 0
    tiempo = 0
    j = 0
    for i in range(len(ruta_camion)-1):
        if ruta_camion[i+1] == ruta_dron[j+1]:
            costo += costosC[ruta_camion[i]][ruta_camion[i+1]]
            tiempo += tiemposC[ruta_camion[i]][ruta_camion[i+1]]
            j += 1
        else:
            costo += costosC[ruta_camion[i]][ruta_camion[i+1]]
            costo += costosD[ruta_dron[j]][ruta_dron[j+1]] + costosD[ruta_dron[j+1]][ruta_dron[j+2]]
            tiempo += max(tiemposC[ruta_camion[i]][ruta_camion[i+1]],tiemposD[ruta_dron[j]][ruta_dron[j+1]] + tiemposD[ruta_dron[j+1]][ruta_dron[j+2]])
            j += 2

    return costo, tiempo

def ejecutar(lista_nodos,hormigas,iteraciones,costosC,costosD,tiemposC,tiemposD,feromonasC,feromonasD,alfa,beta,const,max_vuelo,max_capacidad,limite_tiempo,evaporacion):
    cant_nodos = len(lista_nodos)
    mejor_ruta_camion = []
    mejor_ruta_dron = []
    mejor_costo = 1
    mejor_tiempo = 0
    peso_solucion = 1


    tiempo_inicial = time.time()
    for it in range(iteraciones):
        print(it)
        historia_iteracion = []
        max_fc = max_feromonas(mejor_costo,evaporacion)
        min_fc = min_feromonas(max_fc,cant_nodos,mejor_costo)
        max_fd = max_feromonas(mejor_costo,evaporacion)
        min_fd = min_feromonas(max_fd,cant_nodos,mejor_costo)

        local_ruta_camion = []
        local_ruta_dron = []
        local_costo = 1
        local_tiempo = 0
        local_feromonasC = []
        local_feromonasD = []

        for h in range(hormigas):
            nodo_inicial = h%cant_nodos
            ruta_camion, ruta_dron = hormiga(lista_nodos,nodo_inicial,costosC,costosD,tiemposC,tiemposD,feromonasC,feromonasD,alfa,beta,const,max_vuelo,max_capacidad)
            costo, tiempo = costo_tiempo(ruta_camion,ruta_dron,costosC,costosD,tiemposC,tiemposD)
            feromonasC, feromonasD = feromonas_local(ruta_camion,ruta_dron,feromonasC,feromonasD,evaporacion,min_fc,max_fc,min_fc,min_fd,peso_solucion,costo)

            if local_costo == 1 or costo < local_costo:
                local_ruta_camion = ruta_camion
                local_ruta_dron = ruta_dron
                local_costo = costo
                local_tiempo = tiempo
                local_feromonasC = feromonasC
                local_feromonasD = feromonasD

            if mejor_costo == 1 or costo < mejor_costo:
                mejor_ruta_camion = ruta_camion
                mejor_ruta_dron = ruta_dron
                mejor_costo = costo
                mejor_tiempo = tiempo
                mejor_feromonasC = feromonasC
                mejor_feromonasD = feromonasD
       

        feromonasC, feromonasD = feromonas_global(local_ruta_camion,local_ruta_dron,feromonasC,feromonasD,evaporacion,min_fc,max_fc,min_fd,max_fd,peso_solucion,mejor_costo)


    tiempo_final = time.time()
    return  mejor_ruta_camion, mejor_ruta_dron, mejor_costo, mejor_tiempo, tiempo_final - tiempo_inicial


def limitar_matriz(matriz,limite_min,limite_max):
    for i in range(len(matriz)):
        for j in range(i):
            matriz[i][j] = matriz[j][i] = min(max(matriz[i][j],limite_min),limite_max)
    return matriz

def feromonas_local(ruta_camion,ruta_dron,feromonasC,feromonasD,evaporacion,f_min,f_max,fd_min,fd_max,peso_solucion,costo):
    feromonas_depositadas = peso_solucion/float(costo)
    for x in range(len(ruta_camion)):
        i = ruta_camion[x]
        j = ruta_camion[(x+1)%len(ruta_camion)]
        feromonasC[i][j] = feromonasC[j][i] = feromonas_depositadas
    for i in range(len(ruta_dron)-1):
        i = ruta_dron[x]
        j = ruta_dron[(x+1)%len(ruta_dron)]
        feromonasD[i][j] = feromonasD[j][i] = feromonas_depositadas

    feromonasD = limitar_matriz(feromonasD,fd_min,fd_max)
    feromonasC = limitar_matriz(feromonasC,f_min,f_max)

    return feromonasD, feromonasC

def feromonas_global(ruta_camion,ruta_dron,feromonasC,feromonasD,evaporacion,f_min,f_max,fd_min,fd_max,peso_solucion,costo):
    feromonas_depositadas = peso_solucion/float(costo)
    for i in range(len(feromonasC)):
        for j in range(i):
            feromonasC[i][j] = feromonasC[j][i] = (1-evaporacion)*feromonasC[j][i]
            feromonasD[i][j] = feromonasD[j][i] = (1-evaporacion)*feromonasD[j][i]
    for x in range(len(ruta_camion)-1):
        i = ruta_camion[x]
        j = ruta_camion[x+1]
        feromonasC[i][j] = feromonasC[j][i] = feromonasC[i][j] + feromonas_depositadas

    feromonasD = limitar_matriz(feromonasD,fd_min,fd_max)
    feromonasC = limitar_matriz(feromonasC,f_min,f_max)

    return feromonasC, feromonasD

In [14]:
hormigas= 30
iteraciones= 100
alfa = 0.669
beta = 4.626
const = 5
max_vuelo = 10*60
max_capacidad = 1
limite_tiempo = 1
evaporacion = 0.016



In [15]:
mejor_ruta_camion, mejor_ruta_dron, mejor_costo, mejor_tiempo, tiempo = ejecutar(nodos, hormigas, iteraciones, matriz_costos_camion, matriz_costos_dron,  matriz_tiempos_camion, matriz_tiempos_dron, matriz_feromonas_camion, matriz_fermonas_dron,
         alfa, beta, const, max_vuelo, max_capacidad, limite_tiempo, evaporacion)


0


TypeError: float() argument must be a string or a real number, not 'complex'

In [ ]:
mejor_ruta_camion

In [ ]:
mejor_ruta_dron

In [ ]:
import osmnx as ox
import folium
import folium.plugins as plugins

# Configuración inicial
central_lat, central_lon = -33.456574386393314, -70.65970917320084
# Descarga del grafo de OSMnx
G = ox.graph_from_bbox(north, south, east, west, network_type='drive')

# Calcula las rutas para el camión
calculated_route_segments = []
for i in range(len(mejor_ruta_camion) - 1):
    origen = ox.distance.nearest_nodes(G, float(nodos[mejor_ruta_camion[i]]['long']), float(nodos[mejor_ruta_camion[i]]['lat']))
    destino = ox.distance.nearest_nodes(G, float(nodos[mejor_ruta_camion[i + 1]]['long']), float(nodos[mejor_ruta_camion[i + 1]]['lat']))
    route = ox.shortest_path(G, origen, destino, weight='length')
    route_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route]
    calculated_route_segments.append(route_coords)

# Crea el mapa base
m = folium.Map(location=[central_lat, central_lon], zoom_start=14)

# Dibuja las rutas del camión con flechas
for segment in calculated_route_segments:
    plugins.AntPath(locations=segment, color="blue", weight=5, opacity=0.7, delay=1000).add_to(m)

# Dibuja la ruta del drón con flechas
for i in range(len(mejor_ruta_dron) - 1):
    start_coords = (float(nodos[mejor_ruta_dron[i]]['lat']), float(nodos[mejor_ruta_dron[i]]['long']))
    end_coords = (float(nodos[mejor_ruta_dron[i + 1]]['lat']), float(nodos[mejor_ruta_dron[i + 1]]['long']))

    # Verifica si el dron está viajando en el camión en esta sección
    if mejor_ruta_dron[i] in mejor_ruta_camion and mejor_ruta_dron[i + 1] in mejor_ruta_camion:
        # Encuentra el segmento correspondiente de la ruta del camión
        index_camion_start = mejor_ruta_camion.index(mejor_ruta_dron[i])
        index_camion_end = mejor_ruta_camion.index(mejor_ruta_dron[i + 1])

        # Verifica si los índices son consecutivos en la ruta del camión
        if index_camion_end == index_camion_start + 1:
            segmento_camion = calculated_route_segments[index_camion_start]
            plugins.AntPath(locations=segmento_camion, color="purple", weight=5, opacity=0.7, delay=1000, dash_array='5,5').add_to(m)
        else:
            # Dron y camión no están viajando juntos en esta sección
            plugins.AntPath(locations=[start_coords, end_coords], color="red", weight=5, opacity=0.7, delay=1000).add_to(m)
    else:
        # Estilo de línea con flechas para el dron
        plugins.AntPath(locations=[start_coords, end_coords], color="red", weight=5, opacity=0.7, delay=1000).add_to(m)

# Función para añadir marcadores
def add_marker(m, location, popup_text, delivery_type):
    icon_color = "blue" if delivery_type == "truck" else "red"
    icon_type = "truck" if delivery_type == "truck" else "plane"
    folium.Marker(
        location=location,
        popup=popup_text,
        icon=folium.Icon(icon=icon_type, color=icon_color, prefix='fa')
    ).add_to(m)

# Agrega marcadores para los nodos con información de entrega
for i, nodo in enumerate(nodos):
    location = (float(nodo['lat']), float(nodo['long']))
    if i in mejor_ruta_camion:
        delivery_type = 'truck'
    elif i in mejor_ruta_dron:
        delivery_type = 'drone'
    else:
        continue  # Si el nodo no está en ninguna ruta, no añadir marcador

    popup_text = f"Entrega {i}: {delivery_type.capitalize()}"
    add_marker(m, location, popup_text, delivery_type)

# Alamcane
almacen = (float(nodos[0]['lat']), float(nodos[0]['long']))

# Encuentra la ruta de regreso al almacén para el camión
ultimo_nodo_camion = (float(nodos[mejor_ruta_camion[-1]]['lat']), float(nodos[mejor_ruta_camion[-1]]['long']))
origen = ox.distance.nearest_nodes(G, ultimo_nodo_camion[1], ultimo_nodo_camion[0])
destino = ox.distance.nearest_nodes(G, almacen[1], almacen[0])
ruta_regreso = ox.shortest_path(G, origen, destino, weight='length')
ruta_regreso_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in ruta_regreso]

# Dibuja la ruta de regreso con flechas
if mejor_ruta_dron[-1] == mejor_ruta_camion[-1]:
    # Dron y camión regresan juntos
    for i in range(len(ruta_regreso_coords) - 1):
        plugins.AntPath(locations=[ruta_regreso_coords[i], ruta_regreso_coords[i + 1]], color="blue", weight=5, opacity=0.7, delay=1000).add_to(m)
        plugins.AntPath(locations=[ruta_regreso_coords[i], ruta_regreso_coords[i + 1]], color="purple", weight=5, opacity=0.7, delay=1000, dash_array='5,5').add_to(m)
else:
    # Solo el camión regresa
    plugins.AntPath(locations=ruta_regreso_coords, color="blue", weight=5, opacity=0.7, delay=1000).add_to(m)

# Agrega un marcador para el almacén
folium.Marker(
    location=almacen,
    icon=folium.Icon(color="green", icon="home")
).add_to(m)

# Mostrar el mapa
m


In [ ]:
nodos

In [ ]:
mejor_ruta_camion

In [ ]:
route_coords

In [ ]:
mejor_costo

In [ ]:
mejor_tiempo

In [ ]:
tiempo

In [4]:
import folium
import json

almacen = {"id": "0", "lat": "-33.456574386393314", "long": "-70.65970917320084", "demand": 0}

# Datos de los clientes
with open("nodos.json", 'r', encoding='utf-8') as archivo:
    nodos = json.load(archivo)


nodos.insert(0, almacen)  # Asegúrate de que el almacén es el primer nodo en la lista

# Configuración inicial del mapa
central_lat, central_lon = -33.456574386393314, -70.65970917320084
m = folium.Map(location=[central_lat, central_lon], zoom_start=14)

# Función para añadir marcadores
def add_marker(m, location, popup_text, icon_color='blue', icon='info-sign'):
    folium.Marker(
        location=location,
        popup=popup_text,
        icon=folium.Icon(color=icon_color, icon=icon)
    ).add_to(m)

# Agrega marcadores para los nodos
for nodo in nodos:
    location = (float(nodo['lat']), float(nodo['long']))
    popup_text = f"Nodo {nodo['id']}"
    add_marker(m, location, popup_text)

# Agrega un marcador para el almacén (primer nodo)
almacen = (float(nodos[0]['lat']), float(nodos[0]['long']))
add_marker(m, almacen, 'Almacén', 'green', 'home')

# Mostrar el mapa
m

In [ ]:
import sys
def ejecutar_algoritmo(alfa, beta, const):
    # Define los valores de los parámetros que necesitas para tu algoritmo
    hormigas = 1
    iteraciones = 100
    max_vuelo = 10 * 60  # 10 minutos convertidos a segundos
    max_capacidad = 1
    limite_tiempo = 1
    evaporacion = 0.02

    # Aquí asumo que 'nodos' y las matrices son definidas globalmente o importadas
    mejor_ruta_camion, mejor_ruta_dron, mejor_costo, mejor_tiempo, tiempo = ejecutar(
        nodos, hormigas, iteraciones, matriz_costos_camion, matriz_costos_dron,
        matriz_tiempos_camion, matriz_tiempos_dron, matriz_feromonas_camion, 
        matriz_feromonas_dron, alfa, beta, const, max_vuelo, max_capacidad, 
        limite_tiempo, evaporacion
    )

    # Asumiendo que quieres devolver el mejor costo como resultado
    return mejor_costo
if __name__ == "__main__":
    # Recibe parámetros desde la línea de comandos
    alfa = float(sys.argv[1])
    beta = int(sys.argv[2])
    const = float(sys.argv[3])

    # Llama a la función de tu algoritmo con los parámetros recibidos
    resultado = ejecutar_algoritmo(alfa, beta, const, ...otros parámetros...)
    
    # Imprime el resultado
    print(resultado)

In [ ]:
import sys

if __name__ == "__main__":
    # Recibe parámetros de iRace
    alfa = float(sys.argv[1])
    beta = int(sys.argv[2])
    const = float(sys.argv[3])

    # Ejecuta el algoritmo con los parámetros recibidos
    resultado = ejecutar_algoritmo(alfa, beta, const)

    # Imprime el resultado para que iRace lo utilice
    print(resultado)

In [ ]:
import subprocess
import os

# Cambiar al directorio donde está iRace y tus archivos
os.chdir("C:\\tesis_manrre\\CODIGO FINAL")

# Comando para ejecutar iRace
comando_irace = "irace --scenario scenario.txt"

# Ejecutar iRace
process = subprocess.Popen(comando_irace, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
output, error = process.communicate()

# Mostrar resultados
print("Salida:", output.decode())
print("Error:", error.decode())


In [ ]:
irace --scenario "C:/tesis_manrre/CODIGO FINAL/scenario.txt"
